In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/ELVA/CLASIFICADOR/PeptideBERT/PeptideBERT-master
!pip install -r requirements.txt

/content/drive/MyDrive/ELVA/CLASIFICADOR/PeptideBERT/PeptideBERT-master


In [20]:
import numpy as np

task_name = 'own_data_RFU'

# read data
seqs, values  = [], []
with open('./data/custom_data_rfu.csv', 'r') as f:
    for line in f.readlines()[1:]:
        seq, label = line.strip().split(',')
        seqs.append(seq)
        values.append(float(label))

median_val = np.median(values)
# Discretizar a 0/1
labels = [1 if v >= median_val else 0 for v in values]
MAX_LEN = max(map(len, seqs))

# convert to tokens
mapping = dict(zip(
    ['[PAD]','[UNK]','[CLS]','[SEP]','[MASK]','L',
    'A','G','V','E','S','I','K','R','D','T','P','N',
    'Q','F','Y','M','H','C','W'],
    range(30)
))

pos_data, neg_data = [], []
for i in range(len(seqs)):
    seq = [mapping[c] for c in seqs[i]]
    seq.extend([0] * (MAX_LEN - len(seq)))  # padding to max length
    if labels[i] == 1:
        pos_data.append(seq)
    else:
        neg_data.append(seq)

pos_data = np.array(pos_data)
neg_data = np.array(neg_data)

np.savez(
    f'./data/{task_name}-positive.npz',
    arr_0=pos_data
)
np.savez(
    f'./data/{task_name}-negative.npz',
    arr_0=neg_data
)


In [23]:
!python data/split_augment.py

In [24]:
!python train.py

Device: cuda

==============================        DATA        ==============================
Batch size:  32
Train dataset samples:  383
Validation dataset samples:  43
Test dataset samples:  48
Train dataset batches:  12
Validation dataset batches:  2
Test dataset batches:  2

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 3
wandb: You chose "Don't visualize my results"
wandb: Tracking run with wandb version 0.15.8
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
==============================      TRAINING      =====================